<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li></ul></div>

# Защита персональных данных клиентов

Нужно защитить данные клиентов страховой компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. А также требуется обосновать корректность его работы.

Необходимо защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import scipy as sc
import seaborn as sns
import matplotlib.pyplot as plt
from numpy.linalg import inv

from sklearn.metrics import r2_score

In [2]:
insurance = pd.read_csv('/datasets/insurance.csv')

In [3]:
# получение общей информации о данных
def get_info(df):
    display(df.shape)
    display(df.head())
    display(df.info())
    display(df.describe())
    display(df.isna().sum())

In [4]:
get_info(insurance) 

(5000, 5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

**Описание данных:**
- *признаки:* пол, возраст и зарплата застрахованного, количество членов его семьи;
- *целевой признак:* количество страховых выплат клиенту за последние 5 лет.

**Вывод:** 
- датасет с данными клиентов страховой компании содержит 5000 строк и 5 столбцов;
- пропущенных значений не обнаружено;
- столбцы `Возраст` и `Зарплата` имеют тип данных `float64`, остальные колонки - тип `int`, приведу столбцы с вещественным типом данных к целочисленному;
- для числовых столбцов выведена информация о среднем, стандартном отклонении, квартилях и минимальном и максимальном значениях - по данным характеристикам аномальных значений не наблюдается;
- целевой признак - `Страховые выплаты` (более 75% клиентов страховая выплата не была выплачена ни разу; при этом есть клиенты с пятиразовой выплатой).

In [5]:
# преобразование вещественных столбцов к целочисленным
insurance['Возраст'] = insurance['Возраст'].astype('int')
insurance['Зарплата'] = insurance['Зарплата'].astype('int')

## Умножение матриц

В задании необходимо ответить на вопрос и обосновать решение: `признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?`

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Предсказание линейной регрессии на обучающих данных **не изменится** при умножении матрицы признаков на обратимую матрицу.

**Обоснование:**

Для проверки гипотезы заменю $Х$ на $ХР$ и подставлю в исходые формулы. Получаем:
$$
a_1 = XРw_1
$$
$$
w_1 = ((XP)^T XP)^{-1} (XP)^T y
$$

Нужно доказать, что
$$
a = a_1
$$

Подставим $w_1$ в $a_1$:
$$
a_1 = XР((XP)^T XP)^{-1} (XP)^T y
$$

Раскроем скобки, используя свойство транспонированной матрицы:
$$
(AB)^{Т} = B^{Т}A^{Т}
$$

Получаем:
$$
a_1 = XР(P^TX^T XP)^{-1} P^TX^T y
$$

Раскроем скобки, используя свойство обратной матрицы:
$$
(AB)^{-1} = B^{-1}A^{-1}
$$

Получаем:
$$
a_1 = XР(X^T XP)^{-1} (P^T)^{-1} P^TX^T y
$$

$$
a_1 = XР P^{-1}(X^T X)^{-1} (P^T)^{-1} P^TX^T y
$$

По определению обратной матрицы:
$$
А А^{-1} = E
$$

Получаем:
$$
a_1 = XE (X^T X)^{-1} E X^T y
$$

Произведение матрицы на единичную матрицу равно исходной. Поэтому, выражение выше можно переписать:
$$
a_1 = X(X^T X)^{-1} X^T y
$$

Выражение $(X^T X)^{-1} X^T y$ ничто иное как $w$. Подставим в $a_1$ и получим:
$$
a_1 = Xw = a
$$



## Алгоритм преобразования

**Алгоритм**

Для защиты данных клиентов страховой компании в качестве алгоритма преобразования данных буду использовать умножение исходной матрицы признаков на обратимую матрицу.

**Обоснование**

При умножении на обратимую матрицу каждый вектор матричной системы уравнений выступает как коэффициент перед каждым вектором исходной матрицы, увеличивая или уменьшая признаки, но при этом зависимости остаются прежними и оценка предсказания не меняется.

## Проверка алгоритма

In [6]:
features_train = insurance.drop('Страховые выплаты', axis=1)
target_train = insurance['Страховые выплаты']

In [7]:
# размеры полученных выборок
print(features_train.shape)
print(target_train.shape)

(5000, 4)
(5000,)


In [8]:
# алгоритм преобразования
class Transformer:
    def __init__(self):
        self.random_matrix = None
        
    def fit(self, features):
        n = features.shape[1]
        while True:
            self.random_matrix = np.random.normal(size=(n, n))
            try:
                np.linalg.inv(self.random_matrix)
            except np.linalg.LinAlgError:
                continue
            break
    
    def transform(self, features):
        return np.dot(features.values, self.random_matrix)

In [9]:
transformer = Transformer()
transformer.fit(features_train)
encoded_features = transformer.transform(features_train)

In [10]:
# проверка, что признаки закодировались
print(encoded_features)

[[  7959.86493152 -53657.23177087  23588.61056273 111010.76950075]
 [  6098.59460922 -41120.97231393  18057.15321381  85058.27988759]
 [  3370.0583335  -22728.68103916   9975.01592595  47008.52669979]
 ...
 [  5441.25325005 -36663.72659356  16130.89739608  75866.54829763]
 [  5248.27286273 -35367.56732146  15557.73051635  73181.92185144]
 [  6515.56364054 -43915.84806405  19314.18592473  90863.88260141]]


In [11]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.dot(np.dot(np.linalg.inv(np.dot(X.T, X)), X.T), y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [12]:
# обучение модели линейной регрессии на исходных признаках
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_train)
r2 = r2_score(target_train, predictions)
print(r2)

0.42494550308169177


In [13]:
# обучение модели линейной регрессии на закодированных признаках
transform_features_model = LinearRegression()
transform_features_model.fit(encoded_features, target_train)
transform_features_predictions = transform_features_model.predict(encoded_features)
r2 = r2_score(target_train, transform_features_predictions)
print(r2)

0.4249455030787186


**Вывод:** в результате обучения модели линейной регрессии на исходных и закодированных признаках были получены одинаковые значения коэффициента детерминации R2, что подтверждает теоретический вывод формулы.